In [53]:
from atmodeller import debug_logger
from atmodeller.interfaces import GasSpecies
from atmodeller.interior_atmosphere import Species, Planet, InteriorAtmosphereSystem
from atmodeller.constraints import MassConstraint, IronWustiteBufferConstraintHirschmann, SystemConstraints
from atmodeller.solubilities import BasaltDixonH2O
from atmodeller.utilities import earth_oceans_to_kg

In [54]:
logger = debug_logger()

# No Solubilities

Set up a case where only H2O and H2 are coupled, so CO can only respond by mass balance.

In [55]:
species: Species = Species()
species.append(GasSpecies(formula="H2O"))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
# Don't need to include CO2 to show the effect
#species.append(GasSpecies(formula='CO2'))
species

[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O found in JANAF (JANAF name = H2O)
[10:37:01 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2O using thermodynamic data in JANAF (JANAF name = H2O)
[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 found in JANAF (JANAF name = H2)
[10:37:01 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2 using thermodynamic data in JANAF (JANAF name = H2)
[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = O2
[10:37:01 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for O2 found in JANAF (JANAF name = O2)
[10:37:01 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies O2 using thermodynamic data in JANAF (JANAF n

Species([GasSpecies(formula='H2O', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x12767f050>, name_in_dataset='H2O', _formula=Formula('H2O'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x1054add90>), _output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x12767c210>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(formula='H2', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x1276f5910>, name_in_dataset='H2', _formula=Formula('H2'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x127713610>), _output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x127035b10>, solid_melt_distri

In [56]:
planet: Planet = Planet(surface_temperature=1500, mantle_mass = 3.249E24, surface_radius = 5.804E6, mantle_melt_fraction=1)

[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 3248999999999999788187648.000000
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 5804000.000000
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 4610699520224821698887680.000000
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 1500.000000
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 9.135193
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None


In [57]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[10:37:01 - atmodeller.initial_condition   - INFO     ] - Creating InitialConditionConstant
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Creating a reaction network
[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Species = ['H2O', 'H2', 'O2', 'CO']
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - augmented_matrix = 
[[1. 0. 2. 1. 0. 0. 0.]
 [0. 0. 2. 0. 1. 0. 0.]
 [2. 0. 0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0. 0. 1.]]
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Augmented_matrix after forward elimination = 
[[ 1.   0.   2.   1.   0.   0.   0. ]
 [ 0.   1.  -2.  -1.   0.   0.   1. ]
 [ 0.   0.  -4.  -2.   0.   1.   0. ]
 [ 0.   0.   0.  -1.   1.   0.5  0. ]]
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Augmented_matrix after backward substitution = 
[[ 1.    0.    0.    0.    0.    0.5   0.  ]
 [ 0.    1.    0.    0.    0.   -0.5   1.  ]
 [-

Set your parameters below

In [58]:
# Manually tweaked the parameters to find an example case that demonstrates how the CO pressure can
# increase above 5 orders magnitude
# Dan played around a bit with these parameters, and there doesn't seem to be a simple linear
# relatioship. For example, decreasing the log10_shift to -3 from -2 reduces the CO ratio, and
# increasing the log10_shift to -1 also reduces the CO ratio.
# Would be good to set up an MC with these parameters, and then plot corner plots with
# the ratio = CO(with solubility)/CO(no solubility) to see where the ridge of maximum CO ratio
# is.
H_oceans = 1
CH_ratio = 0.1
log10_shift = -2

In [59]:
mass_H = earth_oceans_to_kg(H_oceans)
mass_C = CH_ratio*mass_H
constraints = SystemConstraints()
constraints.append(MassConstraint(species="H", value=mass_H))
constraints.append(MassConstraint(species='C', value=mass_C))
constraints.append(IronWustiteBufferConstraintHirschmann(log10_shift=log10_shift))

In [60]:
interior_atmosphere.solve(constraints)

[10:37:01 - atmodeller.interior_atmosphere - INFO     ] - Solving system number 0
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Set constraints
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Constraints: [MassConstraint(name='mass', species='H', value=1.5500015377899477e+20),
 MassConstraint(name='mass', species='C', value=1.550001537789948e+19),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-2,
                                       pressure=None)]
[10:37:01 - atmodeller.initial_condition   - DEBUG    ] - InitialConditionConstant: value = 10
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - total_pressure = 30.000000000000025
[10:37:01 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to constraints = [  1.           1.         -13.60049734   1.  

# With H2O solubility

In [61]:
species: Species = Species()
species.append(GasSpecies(formula="H2O", solubility=BasaltDixonH2O()))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
#species.append(GasSpecies(formula='CO2'))
species

[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O found in JANAF (JANAF name = H2O)
[10:37:02 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2O using thermodynamic data in JANAF (JANAF name = H2O)
[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 found in JANAF (JANAF name = H2)
[10:37:02 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2 using thermodynamic data in JANAF (JANAF name = H2)
[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = O2
[10:37:02 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for O2 found in JANAF (JANAF name = O2)
[10:37:02 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies O2 using thermodynamic data in JANAF (JANAF n

Species([GasSpecies(formula='H2O', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x127608750>, name_in_dataset='H2O', _formula=Formula('H2O'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x127646dd0>), _output=None, solubility=<atmodeller.solubilities.BasaltDixonH2O object at 0x127644910>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(formula='H2', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x126e19010>, name_in_dataset='H2', _formula=Formula('H2'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x1277b3dd0>), _output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x1276eafd0>, solid_melt_di

In [62]:
interior_atmosphere_sol: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[10:37:02 - atmodeller.initial_condition   - INFO     ] - Creating InitialConditionConstant
[10:37:02 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[10:37:02 - atmodeller.interior_atmosphere - INFO     ] - Creating a reaction network
[10:37:02 - atmodeller.interior_atmosphere - INFO     ] - Species = ['H2O', 'H2', 'O2', 'CO']
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - augmented_matrix = 
[[1. 0. 2. 1. 0. 0. 0.]
 [0. 0. 2. 0. 1. 0. 0.]
 [2. 0. 0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0. 0. 1.]]
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Augmented_matrix after forward elimination = 
[[ 1.   0.   2.   1.   0.   0.   0. ]
 [ 0.   1.  -2.  -1.   0.   0.   1. ]
 [ 0.   0.  -4.  -2.   0.   1.   0. ]
 [ 0.   0.   0.  -1.   1.   0.5  0. ]]
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Augmented_matrix after backward substitution = 
[[ 1.    0.    0.    0.    0.    0.5   0.  ]
 [ 0.    1.    0.    0.    0.   -0.5   1.  ]
 [-

In [63]:
interior_atmosphere_sol.solve(constraints)

[10:37:02 - atmodeller.interior_atmosphere - INFO     ] - Solving system number 0
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Set constraints
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Constraints: [MassConstraint(name='mass', species='H', value=1.5500015377899477e+20),
 MassConstraint(name='mass', species='C', value=1.550001537789948e+19),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-2,
                                       pressure=None)]
[10:37:02 - atmodeller.initial_condition   - DEBUG    ] - InitialConditionConstant: value = 10
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - total_pressure = 30.000000000000025
[10:37:02 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to constraints = [  1.           1.         -13.60049734   1.  

# Comparison of results

In [64]:
interior_atmosphere.solution_dict # No solubility

{'H2O': 4.709411784734093,
 'H2': 56.08495464139533,
 'O2': 2.5136368338875423e-14,
 'CO': 1.0203723415393124}

In [65]:
interior_atmosphere.total_pressure # bar

61.814738767668764

In [66]:
interior_atmosphere.atmospheric_mean_molar_mass*1000 # g/mol

3.6638969474143765

In [67]:
interior_atmosphere_sol.solution_dict # With solubility

{'H2O': 0.19234607497589729,
 'H2': 2.2958634456133202,
 'O2': 2.5022834039851607e-14,
 'CO': 5.7073602115301405}

In [68]:
interior_atmosphere_sol.total_pressure # bar

8.195569732119383

In [69]:
interior_atmosphere_sol.atmospheric_mean_molar_mass*1000 # g/mol

20.493675500365846

Compute the increase in the CO pressure with H2O solubility versus no solubility.

In [70]:
ratio = interior_atmosphere_sol.solution_dict['CO'] / interior_atmosphere.solution_dict['CO']
ratio

5.593409365565648